# Map Consensus (Experimental Feature)

## Overview

This document describes an **experimental feature** called **Map Consensus**, which explores the idea of **distributed map fusion through neighbor communication**.

The current implementation is a **proof-of-concept (PoC)** intended to test the `combine_maps` mechanism, where each node iteratively fuses its local map with maps received from neighboring nodes in a predefined communication graph.

This idea may be useful in the future for **multi-robot mapping or cooperative SLAM**, but at present it remains an exploratory concept.

---

In [ ]:
from multiprocessing import Pool
from json import dumps, loads

from topolink import NodeHandle


class ExtendedNodeHandle(NodeHandle):
    """
    An extended NodeHandle with experimental features:
    - combine_maps: Combines local maps with neighbor maps by
        1) unioning the keys of all maps
        2) performing weighted averaging of the values for each key
    """

    def __init__(self, idx: str, graph_name: str = "default") -> None:
        super().__init__(idx, graph_name, transport="ipc")

    def combine_maps(
        self, local_map: dict[str, float], alpha: float = 0.5
    ) -> dict[str, float]:
        """
        Combines the local map with neighbor maps using weighted averaging.
        This is an experimental feature and may be useful for multi-robot mapping applications.

        Args:
            local_map (dict[str, float]): A local map to be combined with neighbor maps.

            alpha (float): The weight for neighbor maps in the averaging process. Default is 0.5.

        Returns:
            dict[str, float]: The combined map after weighted averaging.
        """

        serialized_map = dumps(local_map).encode()
        for j_bytes in self._neighbor_idx_bytes:
            self._out_socket.send_multipart([j_bytes, serialized_map], copy=False)

        neighbor_maps: dict[str, dict[str, float]] = {
            j: loads(nc.in_socket.recv()) for j, nc in self._neighbor_contexts.items()
        }

        new_keys = set(local_map.keys())
        for neighbor_map in neighbor_maps.values():
            new_keys = new_keys | neighbor_map.keys()

        combined_map: dict[str, float] = {}
        for key in new_keys:
            local_value = local_map.get(key, 0.0)
            n_values = [n_map.get(key, 0.0) for n_map in neighbor_maps.values()]
            avg_n_value = sum(n_values) / len(n_values)
            new_value = (1 - alpha) * local_value + alpha * avg_n_value

            combined_map[key] = new_value

        return combined_map


def map_consensus(idx: str, map: dict[str, float], n_iter: int = 100) -> None:
    nh = ExtendedNodeHandle(idx)

    alpha = 0.5

    print(f"Node {idx} - Initial map: {map}\n")
    for _ in range(n_iter):
        map = nh.combine_maps(map, alpha)
    print(f"Node {idx} - Final map: {map}\n")


if __name__ == "__main__":
    from topolink import Graph, bootstrap

    NODES = ["1", "2", "3", "4", "5"]
    EDGES = [("1", "2"), ("2", "3"), ("3", "4"), ("4", "5"), ("5", "1")]
    N_NODES = len(NODES)

    graph = Graph(NODES, EDGES, transport="ipc")

    map_for_all = {
        "1": {"[0, 0]": 0.5, "[0, 1]": 0.5, "[1, 0]": 0.0, "[1, 1]": 0.0},
        "2": {"[0, 2]": 0.3, "[0, 3]": 0.7, "[1, 2]": -2.0, "[1, 3]": -1.0},
        "3": {"[2, 0]": 0.2, "[2, 1]": 0.8, "[3, 0]": 2.0, "[3, 1]": 3.0},
        "4": {"[2, 2]": 0.6, "[3, 2]": 0.4, "[2, 3]": 1.0, "[3, 3]": 2.0},
        "5": {"[1, 1]": 0.9, "[1, 2]": 0.1, "[2, 1]": -1.0, "[2, 2]": -0.5},
    }

    with Pool(N_NODES) as pool:
        bootstrap(graph)

        tasks = [
            pool.apply_async(
                map_consensus, args=(str(i + 1), map_for_all.get(str(i + 1), {}))
            )
            for i in range(N_NODES)
        ]
        node_states = [task.get() for task in tasks]

Node 5 - Initial map: {'[1, 1]': 0.9, '[1, 2]': 0.1, '[2, 1]': -1.0, '[2, 2]': -0.5}
Node 2 - Initial map: {'[0, 2]': 0.3, '[0, 3]': 0.7, '[1, 2]': -2.0, '[1, 3]': -1.0}
Node 3 - Initial map: {'[2, 0]': 0.2, '[2, 1]': 0.8, '[3, 0]': 2.0, '[3, 1]': 3.0}
Node 1 - Initial map: {'[0, 0]': 0.5, '[0, 1]': 0.5, '[1, 0]': 0.0, '[1, 1]': 0.0}
Node 4 - Initial map: {'[2, 2]': 0.6, '[3, 2]': 0.4, '[2, 3]': 1.0, '[3, 3]': 2.0}





Node 1 - Final map: {'[2, 3]': 0.20000000000000007, '[3, 1]': 0.5999999999999999, '[0, 2]': 0.06000000000000001, '[1, 2]': -0.38, '[0, 0]': 0.10000000000000006, '[0, 3]': 0.14000000000000007, '[2, 1]': -0.04000000000000001, '[3, 2]': 0.07999999999999999, '[3, 3]': 0.40000000000000013, '[1, 0]': 0.0, '[0, 1]': 0.10000000000000006, '[1, 1]': 0.18, '[2, 0]': 0.039999999999999994, '[3, 0]': 0.40000000000000013, '[2, 2]': 0.020000000000000004, '[1, 3]': -0.20000000000000012}
Node 5 - Final map: {'[2, 3]': 0.20000000000000012, '[3, 1]': 0.5999999999999999, '[0, 2]': 0.06, '[1